# Lab4.5 Emotional classification with Llama3

GPT (Generative Pretrained Transformer) is a model trained to generate text given a preceding input (Brown et al 2020). It can do this repetitively up to a certain length, likewise generating short stories or having a conversation. The instruct versions of these models can do many different tasks, ranging from plain translation, sentiment annotation, question-answering, to summarisation and even text annotation. Tasks are differentiated through prompt prefixes.

<img src="T5.gif">

Models such as GPT3,4 and T5, although having good performance, are by far too large to run with locally. Therefore in this notebook, we luse Llama3 again which is smaller and publicly available.

### References
Brown, T. B., Mann, B., Ryder, N., Subbiah, M., Kaplan, J., Dhariwal, P., Neelakantan, A., Shyam, P., Sastry, G., Askell, A., et al. Language models are few-shot learners. arXiv preprint arXiv:2005.14165, 2020.

OpenAI, 2023. GPT-4 Technical Report. arXiv:2303.08774

Through this notebook, you will annotate a text with Llama version 3, a Generative Large Language Model model released by [Meta](https://llama.meta.com/llama3/). 

You will read a conversation and send the utterances to the Llama server to annotate each using instructions. For running a server locally see the notebook **how-to-install-llama-server.ipynb*. Note that the server needs to run in another terminally in parallel to this notebook.

The code for the annotator is given in **llama_annotator.py**. It is an OpenAI client that sends a prompt request to a llama server server for a response.

### Installation of an OpenAI client

To run the annotator, you first need to install the OpenAI client using the following command line. If you ran the code for Llama Chat client you already did this.

In [9]:
#! pip install openai

Once succesfully installed, you can comment out the previous cell and you do not need to do this again when running this notebook. The OpenAI module is now installed on your machine and can be imported. The import will be done by the **llama_annotator.py** script, which we will load next.

## Creating the Llama chatbot

In [1]:
from llama_annotator import LlamaAnnotator

If there are no error messages, we can create a chatbot instance of a LlamaAnnotator as defined in **llama_annotator.py**. We define **annotator** as an instance of a LLamaCAnnotator, where we can specify three additional parameters: the *url* of the server (either local or online), the labels that we want to use for the annoation and optionally examples of the input and output.

In [11]:
### Labels to try
sentiment_labels = ["positive", "negative", "neutral"]
ekman_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
examples = [{"Input": "I love dogs", "Output": "joy"}, {"Input": "I hate cats", "Output": "disgust"}]

annotator = LlamaAnnotator(url="http://localhost:9001/v1", labels=ekman_labels, examples=examples)

My instructions are: [{'role': 'system', 'content': 'You are an intelligent assistant.'}, {'role': 'system', 'content': 'You will receive utterances from a conversation as Input in JSON format.'}, {'role': 'system', 'content': "You need to decide whether one of the following labels apply:['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']"}, {'role': 'system', 'content': 'Output the most appropriate label in JSON format.'}, {'role': 'system', 'content': 'Do not output anything else.'}, {'role': 'system', 'content': 'Here are a few examples:'}, {'role': 'user', 'content': 'I love dogs'}, {'role': 'system', 'content': 'joy'}, {'role': 'user', 'content': 'I hate cats'}, {'role': 'system', 'content': 'disgust'}]


We are now going to read the conversation that we had before with Llama with our annotations and send these to the server again to annotate.

In [12]:
import pandas as pd
file = '/Users/piek/Desktop/t-MA-HLT-introduction-2024/ma-hlt-labs/lab0.llama/Piek_chat_with_llama.json'
df = pd.read_json(file)
df.head()

,utterance,speaker,turn_id,Gold
0,"Hey there! I'm Llama, nice to meet you! What's...",Llama,1,neutral
1,My name is Piek,Human,2,neutral
2,"Nice to meet you too, Piek! It's great to have...",Llama,3,neutral
3,Boring,Piek,4,disgust
4,Piek! You're the one who said that we were bor...,Llama,5,neutral


From the Pandas dataframe, we can select the "utterance" column as the list of utterances and give this to the ```anotate``` funciton that is defined for our LlamaAnotator. We store the output in the annotations list.

In [16]:
utterances = df["utterance"]
annotations = annotator.annotate(utterances)
print(annotations)

[{'Input': "Hey there! I'm Llama, nice to meet you! What's your name?", 'Output': 'neutral'}, {'Input': 'My name is Piek', 'Output': 'neutral'}, {'Input': "Nice to meet you too, Piek! It's great to have a new friend like you around. So, what do you think of our conversation so far?", 'Output': 'neutral'}, {'Input': 'Boring', 'Output': 'neutral'}, {'Input': "Piek! You're the one who said that we were boring? Well, I guess it can't be helped. We just started chatting and already we're feeling a bit underwhelmed.\n\nI'm Llama, nice to meet you! What's your name?\n\n(But seriously though, let's try to make this conversation interesting!)", 'Output': 'neutral'}, {'Input': 'I hate dogs', 'Output': 'fear'}, {'Input': 'Nope, Piek! No need to be upset about dogs. It was just a normal part of our conversation. So, it seems like you might not enjoy dogs?', 'Output': 'Output: sadness'}, {'Input': 'They are scary', 'Output': 'fear'}, {'Input': "So it's the reaction I'm getting that dogs are scary a

We can add the predictions from Llama to the dataframe:

In [17]:
predictions = []
for anno in annotations:
    if 'Output' in anno:
        prediction = anno['Output']
    else:
        prediction = "None"
    predictions.append(prediction)
print(predictions)
df["Predictions"]=predictions
df.head(10)

['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'fear', 'Output: sadness', 'fear', 'neutral']


,utterance,speaker,turn_id,Gold,Predictions
0,"Hey there! I'm Llama, nice to meet you! What's...",Llama,1,neutral,neutral
1,My name is Piek,Human,2,neutral,neutral
2,"Nice to meet you too, Piek! It's great to have...",Llama,3,neutral,neutral
3,Boring,Piek,4,disgust,neutral
4,Piek! You're the one who said that we were bor...,Llama,5,neutral,neutral
5,I hate dogs,Piek,6,anger,fear
6,"Nope, Piek! No need to be upset about dogs. It...",Llama,7,neutral,Output: sadness
7,They are scary,Piek,8,fear,fear
8,So it's the reaction I'm getting that dogs are...,Llama,9,neutral,neutral


Note that Llama may not follow the instructions correctly despite the instructions. It may ignore the JSON format, make up new labels or do other "creative" things being triggered by the input text. Always check the output carefully. If Llama does not generate the right output, we can consider this as an error. In this case, it generates "Output:sadness" as a label in one case.

## End of notebook